# Ingest Transmission Lines (≥115 kV)

This notebook ingests a public transmission line dataset, standardizes key fields, filters to ≥115 kV assets, validates geometry, and writes a canonical transmission table to PostGIS. The public transmission dataset is incomplete by design. Results are interpreted as indicative coverage only; no attempt is made to infer missing assets.

## Inputs
- Public transmission lines dataset (configured path below)

## Outputs
- PostGIS table: `transmission_lines`
  - Columns: line_id, source, owner, voltage_kv, geom
- Indexes for downstream spatial analysis

### Imports

In [1]:
import os
import re
from pathlib import Path

import pandas as pd
import geopandas as gpd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

### Define Paths

In [2]:
CWD = Path.cwd()
ROOT = CWD.parent if CWD.name.lower() == "notebooks" else CWD

DATA_RAW = ROOT / "data" / "raw"
TX_ROOT = DATA_RAW / "transmission"

load_dotenv(ROOT / ".env")

print("ROOT:", ROOT)
print("DATA_RAW:", DATA_RAW)
print("TX_ROOT:", TX_ROOT)

ROOT: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure
DATA_RAW: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\data\raw
TX_ROOT: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\data\raw\transmission


## Database Connection

In [3]:
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "wildfire_grid")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

missing = [k for k, v in {"DB_USER": DB_USER, "DB_PASSWORD": DB_PASSWORD}.items() if not v]
if missing:
    raise ValueError(f"Missing env vars: {missing}. Check your .env at {ROOT/'.env'}")

DB_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DB_URL)

with engine.begin() as conn:
    conn.execute(text("SELECT 1;"))

print("DB:", DB_NAME, "@", DB_HOST, DB_PORT)

DB: wildfire_grid @ localhost 5432


### Find transmission Dataset File

In [4]:
if not TX_ROOT.exists():
    raise FileNotFoundError(f"Missing transmission root dir: {TX_ROOT}")

candidates = list(TX_ROOT.rglob("*.gpkg")) + list(TX_ROOT.rglob("*.shp"))

if len(candidates) == 0:
    raise FileNotFoundError(
        f"No .gpkg or .shp files found under {TX_ROOT}. "
        "Check that the dataset was downloaded and extracted correctly."
    )

if len(candidates) > 1:
    print("Multiple candidate transmission files found:")
    for p in candidates:
        print(" -", p)
    raise ValueError(
        "Multiple candidates found. Remove extras or hardcode the correct path."
    )

TX_PATH = candidates[0]
print("Using transmission dataset:", TX_PATH)

Using transmission dataset: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\data\raw\transmission\hifld\US_Electric_Power_Transmission_Lines_-5565712113468837121\Electric_Power_Transmission_Lines_A.shp


### Load Transmission Dataset

In [5]:
tx_raw = gpd.read_file(TX_PATH)

if tx_raw.empty:
    raise ValueError("Transmission dataset loaded but contains zero rows.")

print("Rows:", len(tx_raw))
print("CRS:", tx_raw.crs)
print("Geom types:", tx_raw.geom_type.value_counts().to_dict())
print("Columns:", list(tx_raw.columns))
tx_raw.head(3)

Rows: 94619
CRS: EPSG:3857
Geom types: {'LineString': 94478, 'MultiLineString': 141}
Columns: ['OBJECTID_1', 'ID', 'TYPE', 'STATUS', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE', 'OWNER', 'VOLTAGE', 'VOLT_CLASS', 'INFERRED', 'SUB_1', 'SUB_2', 'Shape__Len', 'geometry']


,OBJECTID_1,ID,TYPE,STATUS,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,OWNER,VOLTAGE,VOLT_CLASS,INFERRED,SUB_1,SUB_2,Shape__Len,geometry
0,1,100000,OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, OpenStreetMap",2015-07-07,IMAGERY,2016-02-17,NOT AVAILABLE,-999999.0,NOT AVAILABLE,N,UNKNOWN128553,TAP139917,2250.169338,"LINESTRING (-9397293.05 5271551.21, -9397292.7..."
1,2,100001,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861",2014-05-02,IMAGERY,2020-04-16,ALABAMA POWER CO,115.0,100-161,Y,UNKNOWN109715,TAP142776,19187.282587,"LINESTRING (-9759902.182 3555017.969, -9759682..."
2,3,100002,AC; OVERHEAD,IN SERVICE,221121,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,"IMAGERY, EIA 861, https://www.tva.gov/file_sou...",2014-04-24,IMAGERY,2019-03-11,TENNESSEE VALLEY AUTHORITY,161.0,100-161,Y,UNKNOWN109571,TAP144762,5619.797126,"LINESTRING (-9846204.435 3972849.122, -9846202..."


### Detect Voltage Column & Parse kV

In [6]:
VOLTAGE_CANDIDATES = [
    "VOLTAGE", "VOLT_KV", "VOLTAGE_KV", "KV", "kV", "VOLT",
    "MAX_VOLT", "MAX_KV", "VOLTAGE_K"
]

OWNER_CANDIDATES = [
    "OWNER", "OWNERNAME", "OWNER_NAME", "OPER", "OPERATOR",
    "UTILITY", "COMPANY", "TRANS_OWNER", "PTO"
]

def detect_column(df: pd.DataFrame, candidates: list[str]) -> str | None:
    return next((c for c in candidates if c in df.columns), None)

def parse_voltage_kv(val):
    if val is None or (isinstance(val, float) and pd.isna(val)):
        return None
    if isinstance(val, (int, float)):
        if val <= 0:
            return None
        return float(val)
    s = str(val).strip()
    if not s:
        return None
    m = re.search(r"(\d+(\.\d+)?)", s.replace(",", ""))
    if not m:
        return None
    kv = float(m.group(1))
    return kv if kv > 0 else None

voltage_col = detect_column(tx_raw, VOLTAGE_CANDIDATES)
owner_col = detect_column(tx_raw, OWNER_CANDIDATES)

print("Detected voltage column:", voltage_col if voltage_col else "(none)")
print("Detected owner column:", owner_col if owner_col else "(none)")

Detected voltage column: VOLTAGE
Detected owner column: OWNER


### Standardize & Filter Transmission Lines

In [7]:
if voltage_col is None:
    raise KeyError(
        "Could not detect a voltage column. "
        f"Tried: {VOLTAGE_CANDIDATES}. "
        "Inspect the printed columns above and update VOLTAGE_CANDIDATES or set voltage_col explicitly."
    )

tx = tx_raw.copy()

if tx.crs is None:
    raise ValueError("Transmission CRS is missing. Set the correct CRS before proceeding.")

tx = tx[tx.geometry.notna()].copy()
tx = tx[tx.geom_type.isin(["LineString", "MultiLineString"])].copy()

tx["voltage_kv"] = tx[voltage_col].apply(parse_voltage_kv)
tx = tx[tx["voltage_kv"].notna()].copy()
tx = tx[tx["voltage_kv"] >= 115].copy()

if owner_col is not None:
    tx["owner"] = tx[owner_col].astype("string")
else:
    tx["owner"] = pd.Series([None] * len(tx), dtype="string")

tx["source"] = "Public Transmission Lines"

print("Rows after voltage filter (≥115 kV):", len(tx))
tx[["owner", "voltage_kv"]].head(3)

Rows after voltage filter (≥115 kV): 51155


,owner,voltage_kv
1,ALABAMA POWER CO,115.0
2,TENNESSEE VALLEY AUTHORITY,161.0
5,BALTIMORE GAS & ELECTRIC CO,115.0


## Build California Version

In [8]:
tx = tx.to_crs(epsg=4326)

CA_LON_MIN, CA_LON_MAX = -124.48, -114.13
CA_LAT_MIN, CA_LAT_MAX = 32.53, 42.01

n_before = len(tx)
tx = tx.cx[CA_LON_MIN:CA_LON_MAX, CA_LAT_MIN:CA_LAT_MAX].copy()

print("Rows before CA bbox filter:", n_before)
print("Rows after  CA bbox filter:", len(tx))

Rows before CA bbox filter: 51155
Rows after  CA bbox filter: 2167


## Build Transmission GeoDataFrame

In [10]:
tx = tx.reset_index(drop=True)
tx["line_id"] = tx.index.astype("int64") + 1

tx_out = gpd.GeoDataFrame(
    tx[["line_id", "source", "owner", "voltage_kv"]].copy(),
    geometry=tx.geometry,
    crs="EPSG:4326",
)

tx_out = tx_out.rename_geometry("geom")

print("Canonical transmission rows:", len(tx_out))
tx_out.head(3)

Canonical transmission rows: 2167


,line_id,source,owner,voltage_kv,geom
0,1,Public Transmission Lines,BONNEVILLE POWER ADMINISTRATION,500.0,"LINESTRING (-121.31658 42.0061, -121.31725 42...."
1,2,Public Transmission Lines,PORTLAND GENERAL ELECTRIC CO,500.0,"LINESTRING (-121.31658 42.0061, -121.31665 42...."
2,3,Public Transmission Lines,PACIFICORP,500.0,"LINESTRING (-121.31658 42.0061, -121.31603 42...."


## Validation

In [11]:
print("Rows:", len(tx_out))
print("CRS:", tx_out.crs)
print("Null geom:", int(tx_out["geom"].isna().sum()))
print("Invalid geometries:", int((~tx_out.is_valid).sum()))

display(tx_out["voltage_kv"].describe())
display(tx_out["owner"].value_counts(dropna=False).head(15))

Rows: 2167
CRS: EPSG:4326
Null geom: 0
Invalid geometries: 0


count    2167.000000
mean      176.973696
std        90.499689
min       115.000000
25%       115.000000
50%       120.000000
75%       230.000000
max      1000.000000
Name: voltage_kv, dtype: float64

owner
PACIFIC GAS & ELECTRIC COMPANY             892
NOT AVAILABLE                              353
SOUTHERN CALIFORNIA EDISON                 212
NEVADA POWER COMPANY                       137
AZUSA LIGHT & POWER                        126
SIERRA PACIFIC POWER COMPANY                69
SACRAMENTO MUNICIPAL UTILITY DISTRICT       61
SAN DIEGO GAS & ELECTRIC                    59
PACIFICORP                                  37
LOS ANGELES DEPARTMENT OF WATER & POWER     37
LADWP                                       26
IMPERIAL IRRIGATION DISTRICT                25
REDDING ELECTRIC UTILITY                    11
VALLEY ELECTRIC ASSOCIATION COOPERATIVE     10
MERCED IRRIGATION DISTRICT                   9
Name: count, dtype: Int64

## Write to PostGIS

In [12]:
TABLE = "transmission_lines"

with engine.begin() as conn:
    conn.execute(text(f"DROP TABLE IF EXISTS {TABLE};"))

tx_out.to_postgis(
    name=TABLE,
    con=engine,
    if_exists="replace",
    index=False,
)

with engine.begin() as conn:
    conn.execute(text(f"ALTER TABLE {TABLE} ADD PRIMARY KEY (line_id);"))
    conn.execute(text(f"CREATE INDEX IF NOT EXISTS idx_{TABLE}_geom_gist ON {TABLE} USING GIST (geom);"))
    conn.execute(text(f"CREATE INDEX IF NOT EXISTS idx_{TABLE}_voltage ON {TABLE} (voltage_kv);"))
    conn.execute(text(f"CREATE INDEX IF NOT EXISTS idx_{TABLE}_owner ON {TABLE} (owner);"))
    conn.execute(text(f"ANALYZE {TABLE};"))

print(f"Wrote PostGIS table: {TABLE} (rows={len(tx_out):,})")

Wrote PostGIS table: transmission_lines (rows=2,167)


## Preview

In [13]:
summary = pd.read_sql(
    f"""
    SELECT
      COUNT(*) AS n_lines,
      MIN(voltage_kv) AS min_kv,
      MAX(voltage_kv) AS max_kv
    FROM {TABLE};
    """,
    engine
)
display(summary)

preview = pd.read_sql(
    f"SELECT line_id, owner, voltage_kv FROM {TABLE} ORDER BY line_id LIMIT 10;",
    engine
)
preview

,n_lines,min_kv,max_kv
0,2167,115.0,1000.0


,line_id,owner,voltage_kv
0,1,BONNEVILLE POWER ADMINISTRATION,500.0
1,2,PORTLAND GENERAL ELECTRIC CO,500.0
2,3,PACIFICORP,500.0
3,4,PACIFICORP,230.0
4,5,PACIFICORP,500.0
5,6,PACIFICORP,230.0
6,7,PACIFICORP,230.0
7,8,PACIFICORP,230.0
8,9,PACIFICORP,115.0
9,10,PACIFICORP,115.0
